#### <span style="color:#AD07FF"> In this file we will parse json file and convert it to pandas dataframe <br> Depending on which algorithm we want to do, It is possible not to use all variables. <br>  For example, in our case, we do not think it is necessary to know <br> the quantity of nutrients the customer got after each meal. 

# <span style="color:#D75B08"><div align="center">**Imports** 

In [3]:
import json
import itertools
import numpy as np
import pandas as pd
from time import time
import csv

import warnings
warnings.filterwarnings("ignore")

# <span style="color:#D75B08"><div align="center">**Read Data** 

In [4]:
data = pd.read_csv('../data/mfp-diaries.tsv', sep='\t')
data.head()

,1,2014-09-14,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"": [{""name"": ""Calories"", ""value"": ""412""}, {""name"": ""Carbs"", ""value"": ""29""}, {""name"": ""Fat"", ""value"": ""24""}, {""name"": ""Protein"", ""value"": ""21""}, {""name"": ""Sodium"", ""value"": ""258""}, {""name"": ""Sugar"", ""value"": ""29""}], ""name"": ""my - McDonalds Espresso Pronto\u00ae Flat White, 2 TALL""}, {""nutritions"": [{""name"": ""Calories"", ""value"": ""170""}, {""name"": ""Carbs"", ""value"": ""25""}, {""name"": ""Fat"", ""value"": ""5""}, {""name"": ""Protein"", ""value"": ""20""}, {""name"": ""Sodium"", ""value"": ""260""}, {""name"": ""Sugar"", ""value"": ""2""}], ""name"": ""Quest Bar - Banana Nut Muffin Natural Protein Bar, 60 g""}, {""nutritions"": [{""name"": ""Calories"", ""value"": ""176""}, {""name"": ""Carbs"", ""value"": ""33""}, {""name"": ""Fat"", ""value"": ""1""}, {""name"": ""Protein"", ""value"": ""5""}, {""name"": ""Sodium"", ""value"": ""195""}, {""name"": ""Sugar"", ""value"": ""0""}], ""name"": ""Uncle Tobys Australia - Vita Brits, 3 Biscuits 33.3g""}, {""nutritions"": [{""name"": ""Calories"", ""value"": ""342""}, {""name"": ""Carbs"", ""value"": ""34""}, {""name"": ""Fat"", ""value"": ""12""}, {""name"": ""Protein"", ""value"": ""24""}, {""name"": ""Sodium"", ""value"": ""402""}, {""name"": ""Sugar"", ""value"": ""34""}], ""name"": ""Pauls - Smarter White Milk, 600 ml""}, {""nutritions"": [{""name"": ""Calories"", ""value"": ""180""}, {""name"": ""Carbs"", ""value"": ""22""}, {""name"": ""Fat"", ""value"": ""7""}, {""name"": ""Protein"", ""value"": ""21""}, {""name"": ""Sodium"", ""value"": ""310""}, {""name"": ""Sugar"", ""value"": ""1""}], ""name"": ""Quest Bar - Cookies and Cream, 1 bar""}, {""nutritions"": [{""name"": ""Calories"", ""value"": ""180""}, {""name"": ""Carbs"", ""value"": ""46""}, {""name"": ""Fat"", ""value"": ""0""}, {""name"": ""Protein"", ""value"": ""0""}, {""name"": ""Sodium"", ""value"": ""100""}, {""name"": ""Sugar"", ""value"": ""22""}], ""name"": ""Gu Sports - Chomps, 8 pieces""}, {""nutritions"": [{""name"": ""Calories"", ""value"": ""170""}, {""name"": ""Carbs"", ""value"": ""25""}, {""name"": ""Fat"", ""value"": ""5""}, {""name"": ""Protein"", ""value"": ""20""}, {""name"": ""Sodium"", ""value"": ""260""}, {""name"": ""Sugar"", ""value"": ""2""}], ""name"": ""Quest Bar - Banana Nut Muffin Natural Protein Bar, 60 g""}, {""nutritions"": [{""name"": ""Calories"", ""value"": ""160""}, {""name"": ""Carbs"", ""value"": ""25""}, {""name"": ""Fat"", ""value"": ""5""}, {""name"": ""Protein"", ""value"": ""20""}, {""name"": ""Sodium"", ""value"": ""240""}, {""name"": ""Sugar"", ""value"": ""2""}], ""name"": ""Quest - Protein Bar Chocolate Peanut Butter, 1 bar (60 g)""}, {""nutritions"": [{""name"": ""Calories"", ""value"": ""686""}, {""name"": ""Carbs"", ""value"": ""59""}, {""name"": ""Fat"", ""value"": ""31""}, {""name"": ""Protein"", ""value"": ""40""}, {""name"": ""Sodium"", ""value"": ""1,105""}, {""name"": ""Sugar"", ""value"": ""9""}], ""name"": ""Grilld - Simply Grilled (Real), 1.25 burger""}, {""nutritions"": [{""name"": ""Calories"", ""value"": ""252""}, {""name"": ""Carbs"", ""value"": ""27""}, {""name"": ""Fat"", ""value"": ""14""}, {""name"": ""Protein"", ""value"": ""5""}, {""name"": ""Sodium"", ""value"": ""352""}, {""name"": ""Sugar"", ""value"": ""1""}], ""name"": ""Grilld - Snack Chips, 1 snack size""}, {""nutritions"": [{""name"": ""Calories"", ""value"": ""196""}, {""name"": ""Carbs"", ""value"": ""15""}, {""name"": ""Fat"", ""value"": ""10""}, {""name"": ""Protein"", ""value"": ""10""}, {""name"": ""Sodium"", ""value"": ""176""}, {""name"": ""Sugar"", ""value"": ""7""}], ""name"": ""Tasti - Salted Caramel Protein Bar, 40 g""}], ""sequence"": 1}]","{""total"": [{""name"": ""Calories"", ""value"": 2924}, {""name"": ""Carbs"", ""value"": 340}, {""name"": ""Fat"", ""value"": 114}, {""name"": ""Protein"", ""value"": 186}, {""name"": ""Sodium"", ""value"": 3658}, {""n

# <span style="color:#D75B08"><div align="center">**Create Dictionary of unique foods** 

#### <span style="color:#D75B08"> As we can see, for each user, we have the names of their eaten food <br> but instead of this ,we want the numbers , so we will take the unique name foods to a separate dictionary <br> and food name replaces through the corresponding unique ID

In [5]:
# parameters: name of dishes, dictionary
# this method adds food if it doesn't exists
def add_food(food,all_foods):
    global idx
    if food not in all_foods:
        all_foods[food] = idx
        idx+=1
        
# one person may eat max 6 times a day
# This method will make unique foods dictionary
def put_all_food_names(food_entries,all_foods):
    for food_entrie in json.loads(food_entries): 
        for food in food_entrie["dishes"]:
            add_food(food["name"].split(",")[0],all_foods)
            
            
# This method gets an information about daily goals and total received calories and other nutrients
def get_agregate_intake_goal(agregate_intake_goal):
    res = []
    for nutritient in agregate_intake_goal["total"]:
        res.append(nutritient["value"])
    for nutritient in agregate_intake_goal["goal"]:
        res.append(nutritient["value"])
    return res

In [6]:
food_enumeration = {} # Dictionary where will be unique foods. Key will be name and value id
idx = 1 # ID inumeration start point

test_dataframe = data.copy()
start = time()

test_dataframe.iloc[:,2].apply(lambda elem: put_all_food_names(elem,food_enumeration))

print("elapsed time - " + str(time()-start))
print("length of all food - " + str(len(food_enumeration)))

elapsed time - 122.95582461357117
length of all food - 628878


# <span style="color:#D75B08"><div align="center">**Create Apropriate Dataframes** 

#### <span style="color:#D75B08"> Each user may have many records per day <br> so we will merge whole day's records. for example if a customer has a 5 days of record we will have 5 rows on it

In [7]:
# food_ids contains food Id's eaten in particular meal and you can check names in all_foods
def parse_new_applicants(row,all_foods):
    global parsed_df
    values = []
    values.append(row[0]) # user_id 
    values.append(row[1]) # date
    
    food_entries = json.loads(row[2])
    agregate_intake_goal = get_agregate_intake_goal(json.loads(row[3]))
    
    food_ids = []
    for food_entrie in food_entries:
        food_names = [food["name"].split(",")[0] for food in food_entrie["dishes"]]
        food_ids_new = [all_foods[food_name] for food_name in food_names]
        food_ids+=food_ids_new
        
    values.append(food_entrie["sequence"])
    values.append(food_ids)
    values += agregate_intake_goal

    dictionary = dict(zip(df_columns, values))
    parsed_df.append(dictionary)

In [8]:
start = time()

# column names for new dataframe
df_columns = ['user_id','date','sequence','food_ids',
           'total_calories','total_carbs','total_fat','total_protein','total_sodium','total_sugar',
           'goal_calories','goal_carbs','goal_fat','goal_protein','goal_sodium','goal_sugar']


# This global variable will be feed with parsed data
parsed_df = []

test_dataframe = data
_ = test_dataframe.apply(lambda row : parse_new_applicants(row,food_enumeration), axis = 1) 
parsed_df = pd.DataFrame.from_dict(parsed_df)

end = time()
print("elapsed time - " + str(end-start))

elapsed time - 369.07988715171814


# <span style="color:#D75B08"><div align="center">**Save parsed Dataframes** 

In [9]:
# exchanging key and value to use food with ID (key)
food_enumeration = {value:key for key, value in food_enumeration.items()}

# save food name and ID as a new json file , If we need it, we won't have to always generate it 
with open('../data/foods.json', 'w') as fp:
    json.dump(food_enumeration, fp)

In [10]:
parsed_df.to_csv(r'../data/myFitnessPal_parsed.csv',index = False)